In [ ]:
# compiling all files into a big .TextGrid (6 .TextGrid files)

from praatio import tgio
from functools import wraps


filenames = ['game_885-1785s_UPD.TextGrid', 'game_1785-2605s_UPD.TextGrid', 'game_2605-3500s_UPD.TextGrid', 'game_3500-4400_UPD.TextGrid', 'game_4400-5300_UPD.TextGrid']

a = tgio.openTextgrid('game_1-885s_U.TextGrid')
for file in filenames:
    b = tgio.openTextgrid(file)
    combinedTG = a.appendTextgrid(b, True)
combinedTG.save("all.TextGrid")

In [ ]:
# an empty database for the ordered databes of replics

import sqlite3

conn = sqlite3.connect('ordered.db')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS ordered(start_point float, replic text, speaker integer, language text, inter integer)')
conn.close()

In [ ]:
# a database of replics, unordered
# structure: starting_point - replic - speaker - language - inter CS


def replics_db():
    conn = sqlite3.connect('replics.db')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS replics(start_point float, replic text, speaker integer, language text, inter integer)')
#     a list for future intersentenrional code-switches' starting point
    cster_list = []
    tg = tgio.openTextgrid('all.TextGrid')
#     a grid of intersententional code-switches
    cs_inter = tg.tierDict[tg.tierNameList[20]]
    cster_list.extend([entry[0] for entry in cs_inter.entryList])
    for i in range (0, 16, 5):
        starting_point = []
        replics = []
#         a grid with replics
        a = tg.tierDict[tg.tierNameList[i]]
#      a grid with russian words
        rus = tg.tierDict[tg.tierNameList[i + 1]]
        ykt = tg.tierDict[tg.tierNameList[i + 2]]
        rus_full_s = [entry[0] for entry in rus.entryList]
        rus_full_e = [entry[1] for entry in rus.entryList]
        ykt_full_s = [entry[0] for entry in ykt.entryList]
        ykt_full_e = [entry[1] for entry in ykt.entryList]

#         differentiating between code-switches
        for s_point, e_point, label in a.entryList:
            ter = 0
            if s_point in cster_list:
                ter = 1
            if s_point in rus_full_s and e_point in rus_full_e:
                language = 'R'
            elif s_point in ykt_full_s and e_point in ykt_full_e:
                language = 'Y'
            elif s_point in rus_full_s and e_point in ykt_full_e:
                language = 'CS'
            elif s_point in ykt_full_s and e_point in rus_full_e:
                language = 'CS'
            else:
                language = 'N'
            c.execute('INSERT INTO replics VALUES (?, ?, ?, ?, ?)', (s_point, label, i, language, ter))
            
        conn.commit()
       
    c.execute('ATTACH DATABASE "ordered.db" as ordered')
    c.execute('INSERT INTO ordered SELECT * FROM replics ORDER BY start_point ASC')
    conn.commit()  

    conn.close()
    
replics_db()    